<a href="https://colab.research.google.com/github/lcbjrrr/quant/blob/master/J_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Topic:** AI/ML

**Title:** Neural Network (with Flux)

**Author:** Luiz Barboza

**Date:** 20/dec/22

**Lang:** Julia

**Site:** https://quant-research.group/

**Email:** contato@quant-research.group


# Julia Installation

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.8.3" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  nvidia-smi -L &> /dev/null && export GPU=1 || export GPU=0
  if [ $GPU -eq 1 ]; then
    JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.8.3 on the current Colab Runtime...
2022-12-21 20:55:19 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.8/julia-1.8.3-linux-x86_64.tar.gz [130030846/130030846] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...


In [1]:
versioninfo()

Julia Version 1.8.3
Commit 0434deb161e (2022-11-14 20:14 UTC)
Platform Info:
  OS: Linux (x86_64-linux-gnu)
  CPU: 2 × AMD EPYC 7B12
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-13.0.1 (ORCJIT, znver2)
  Threads: 2 on 2 virtual cores
Environment:
  LD_LIBRARY_PATH = /usr/local/nvidia/lib:/usr/local/nvidia/lib64
  LD_PRELOAD = /usr/lib/x86_64-linux-gnu/libtcmalloc.so.4
  JULIA_NUM_THREADS = 2


# Neural Networks (with Flux)

In [73]:
import Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("Lathe") 
Pkg.add("Flux") 

using CSV
using DataFrames
using Lathe
using Lathe.preprocess: TrainTestSplit
using Flux
using Flux: crossentropy, throttle

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.8/Project.toml`
  No Changes to `~/.julia/environments/v1.8/Manifest.toml`


## NN - Classification

In [ ]:
;wget https://raw.githubusercontent.com/lcbjrrr/data/main/gender%20-%20all.csv

In [4]:
#read csv
df=CSV.read("gender - all.csv", DataFrame)
first(df,5)

,G,H,W
,Int64,Float64,Float64
1,0,187.571,109.952
2,0,174.706,73.7775
3,0,188.24,96.7004
4,0,182.197,100.019
5,0,177.5,93.7954


In [6]:
train, test = TrainTestSplit(df,.70)

(7016×3 DataFrame
  Row │ G      H        W        
      │ Int64  Float64  Float64  
──────┼──────────────────────────
    1 │     0  187.571  109.952
    2 │     0  174.706   73.7775
    3 │     0  182.197  100.019
    4 │     0  177.5     93.7954
    5 │     0  170.823   69.1873
    6 │     0  174.714   83.6036
    7 │     0  173.605   76.3505
    8 │     0  170.228   79.9679
    9 │     0  161.179   71.0908
   10 │     0  181.968   97.1551
   11 │     0  164.506   75.967
  ⋮   │   ⋮       ⋮        ⋮
 7007 │     1  154.826   43.4939
 7008 │     1  159.097   46.7516
 7009 │     1  157.678   63.0417
 7010 │     1  152.477   44.4034
 7011 │     1  150.11    50.2408
 7012 │     1  168.079   62.1716
 7013 │     1  170.351   77.6672
 7014 │     1  162.225   58.3979
 7015 │     1  175.347   74.4784
 7016 │     1  157.338   51.6587
                6995 rows omitted, 2984×3 DataFrame
  Row │ G      H        W       
      │ Int64  Float64  Float64 
──────┼─────────────────────────
    1 │   

In [64]:
X_train  = [ [row.H, row.W] for row in eachrow(train)]
y_train = [ [row.G] for row in eachrow(train)]
y_train

7016-element Vector{Vector{Int64}}:
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 ⋮
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]

In [101]:
zip(X_train,y_train)

zip([[187.5714232, 109.9516196], [174.7060363, 73.77748751], [182.1966851, 100.0193047], [177.4997615, 93.79536392], [170.8226598, 69.18734353], [174.7141064, 83.60358573], [173.6052294, 76.35050477], [170.2281321, 79.96792745], [161.1794947, 71.09076199], [181.967645, 97.15507704]  …  [154.825914, 43.49394288], [159.0971543, 46.75161964], [157.6776925, 63.04167285], [152.4773018, 44.4033783], [150.1095551, 50.24076622], [168.0785365, 62.17157008], [170.3505728, 77.66722995], [162.2247002, 58.39787217], [175.3469776, 74.47839152], [157.3383845, 51.65868303]], [[0], [0], [0], [0], [0], [0], [0], [0], [0], [0]  …  [1], [1], [1], [1], [1], [1], [1], [1], [1], [1]])

In [130]:
nn = Chain(
  Dense(2,4, relu),
  Dense(4,3, relu),
  Dense(3,1,identity),
  softmax
  )
L(x,y)=Flux.crossentropy(nn(x), y)
Flux.train!(L,Flux.params(nn),zip(X_train,y_train),ADAM())

In [131]:
pred_train  = [ nn(row[1]) for row in eachrow(X_train)]

7016-element Vector{Vector{Float64}}:
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 ⋮
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]
 [1.0]

In [132]:
function accuracy(y,pred)
  acc = sum(pred.==y)/length(y)
  return acc
end

acc_train = accuracy(y_train, pred_train)
print("Acc. Train: ",acc_train)

Acc. Train: 0.5

In [133]:
X_test  = [ [row.H, row.W] for row in eachrow(test)]
y_test = [ [row.G] for row in eachrow(test)]
pred_test  = [ nn(row[1]) for row in eachrow(X_test)]
acc_test = accuracy(y_test, pred_test)
print("Acc. Test: ",acc_test)

Acc. Test: 0.5



```
batches = (Flux.batch(X_train),Flux.batch(y_train))
nn = Chain(
  Dense(2,4, relu),
  Dense(4,3, relu),
  Dense(3,1,identity),
  softmax
  )
L(x,y)=Flux.crossentropy(nn(x), y)
evalcb = () -> @show(L) 
for i in 1:11
  Flux.train!(L,Flux.params(nn),[batches],ADAM(), cb = Flux.throttle(evalcb, 1))
end
```

